# Create Feature_Usage Objects in ADLSTE_COA 

In [69]:
%connect name=TranscendAWS

Password ············


Success: 'TranscendAWS' connection established


<br/><br/><br/>
## Tables

In [ ]:
--- TBD: need Mr. Carty to move over 

<br/><br/><br/>
## Feature_Usage_LOG  sql for COA

In [70]:
Select trim(substr(infoData,1,8)) as DBSVersion from dbc.dbcinfo where InfoKey = 'VERSION'

DBSVersion
16.20.43


In [47]:
drop table FeatureMap

Success: 0 rows affected

In [82]:

/*{{save:feature_department.csv}}*/
SELECT
 '{siteid}' (VARCHAR(100)) as SiteID
,A.LogDate as LogDate
,u.User_Bucket
,u.User_Department
,count(*) as Request_Count
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -016)))) AS bit016  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -017)))) AS bit017  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -018)))) AS bit018  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -019)))) AS bit019  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -020)))) AS bit020  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -021)))) AS bit021  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -022)))) AS bit022  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -023)))) AS bit023  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -024)))) AS bit024  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -025)))) AS bit025  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -026)))) AS bit026  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -027)))) AS bit027  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -028)))) AS bit028  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -029)))) AS bit029  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -030)))) AS bit030  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -031)))) AS bit031  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -032)))) AS bit032  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -033)))) AS bit033  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -034)))) AS bit034  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -035)))) AS bit035  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -036)))) AS bit036  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -037)))) AS bit037  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -038)))) AS bit038  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -039)))) AS bit039  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -040)))) AS bit040  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -041)))) AS bit041  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -042)))) AS bit042  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -043)))) AS bit043  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -044)))) AS bit044  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -045)))) AS bit045  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -046)))) AS bit046  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -047)))) AS bit047  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -048)))) AS bit048  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -049)))) AS bit049  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -050)))) AS bit050  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -051)))) AS bit051  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -052)))) AS bit052  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -053)))) AS bit053  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -054)))) AS bit054  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -055)))) AS bit055  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -056)))) AS bit056  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -057)))) AS bit057  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -058)))) AS bit058  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -059)))) AS bit059  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -060)))) AS bit060  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -061)))) AS bit061  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -062)))) AS bit062  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -063)))) AS bit063  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -064)))) AS bit064  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -065)))) AS bit065  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -066)))) AS bit066  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -067)))) AS bit067  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -068)))) AS bit068  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -069)))) AS bit069  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -070)))) AS bit070  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -071)))) AS bit071  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -072)))) AS bit072  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -073)))) AS bit073  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -074)))) AS bit074  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -075)))) AS bit075  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -076)))) AS bit076  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -077)))) AS bit077  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -078)))) AS bit078  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -079)))) AS bit079  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -080)))) AS bit080  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -081)))) AS bit081  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -082)))) AS bit082  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -083)))) AS bit083  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -084)))) AS bit084  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -085)))) AS bit085  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -086)))) AS bit086  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -087)))) AS bit087  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -088)))) AS bit088  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -089)))) AS bit089  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -090)))) AS bit090  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -091)))) AS bit091  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -092)))) AS bit092  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -093)))) AS bit093  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -094)))) AS bit094  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -095)))) AS bit095  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -096)))) AS bit096  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -097)))) AS bit097  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -098)))) AS bit098  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -099)))) AS bit099  
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -100)))) AS bit100 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -101)))) AS bit101 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -102)))) AS bit102 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -103)))) AS bit103 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -104)))) AS bit104 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -105)))) AS bit105 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -106)))) AS bit106 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -107)))) AS bit107 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -108)))) AS bit108 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -109)))) AS bit109 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -110)))) AS bit110 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -111)))) AS bit111 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -112)))) AS bit112 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -113)))) AS bit113 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -114)))) AS bit114 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -115)))) AS bit115 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -116)))) AS bit116 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -117)))) AS bit117 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -118)))) AS bit118 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -119)))) AS bit119 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -120)))) AS bit120 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -121)))) AS bit121 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -122)))) AS bit122 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -123)))) AS bit123 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -124)))) AS bit124 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -125)))) AS bit125 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -126)))) AS bit126 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -127)))) AS bit127 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -128)))) AS bit128 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -129)))) AS bit129 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -130)))) AS bit130 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -131)))) AS bit131 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -132)))) AS bit132 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -133)))) AS bit133 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -134)))) AS bit134 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -135)))) AS bit135 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -136)))) AS bit136 
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -137)))) AS bit137 
FROM PDCRINFO.DBQLOGTBL_HST as A
JOIN dim_user u
  on a.UserName = u.UserName
WHERE LogDate BETWEEN date-3 and date
GROUP BY 1,2,3,4;



SiteID,LogDate,User_Bucket,User_Department,Request_Count,bit016,bit017,bit018,bit019,bit020,bit021,bit022,bit023,bit024,bit025,bit026,bit027,bit028,bit029,bit030,bit031,bit032,bit033,bit034,bit035,bit036,bit037,bit038,bit039,bit040,bit041,bit042,bit043,bit044,bit045,bit046,bit047,bit048,bit049,bit050,bit051,bit052,bit053,bit054,bit055,bit056,bit057,bit058,bit059,bit060,bit061,bit062,bit063,bit064,bit065,bit066,bit067,bit068,bit069,bit070,bit071,bit072,bit073,bit074,bit075,bit076,bit077,bit078,bit079,bit080,bit081,bit082,bit083,bit084,bit085,bit086,bit087,bit088,bit089,bit090,bit091,bit092,bit093,bit094,bit095,bit096,bit097,bit098,bit099,bit100,bit101,bit102,bit103,bit104,bit105,bit106,bit107,bit108,bit109,bit110,bit111,bit112,bit113,bit114,bit115,bit116,bit117,bit118,bit119,bit120,bit121,bit122,bit123,bit124,bit125,bit126,bit127,bit128,bit129,bit130,bit131,bit132,bit133,bit134,bit135,bit136,bit137
{siteid},2020-06-10,Unknown,Unknown,690325,192,474302,375552,91768,474316,0,474479,2791,0,18585,3090,990,1077,2013,0,0,0,0,97,24059,171,3843,199742,1268,486635,113,4,14,0,0,83,0,84,4,478,2809,662,2,564,0,0,1887,2009,15790,690295,0,690295,0,0,0,0,690295,690295,0,414,0,0,0,0,15,0,144202,0,0,0,6,638,4347,474479,300882,0,89,316505,0,380469,92246,0,1511,0,0,0,2860,1,12,67,101,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0,5974,0,49641,50203,65,0,0,0,0,0,0,0,0,0,963,657,0,0,0,0,71
{siteid},2020-06-08,TDInternal,dept,39519,0,18569,13733,5268,18569,0,18569,0,0,48,553,553,553,0,0,0,0,0,0,6270,48,48,36356,192,38678,356,456,0,0,0,0,0,0,0,0,384,0,24,0,0,0,0,0,0,39519,0,39519,0,0,0,0,39519,39519,0,24,0,0,0,0,0,0,38324,0,0,0,720,251,251,18569,1465,0,0,606,0,9653,9969,0,0,0,0,0,98,0,0,0,36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,308,0,7103,7155,0,0,0,0,0,0,0,0,0,0,0,110,0,0,0,0,0
{siteid},2020-06-08,Unknown,Unknown,700246,230,463386,371935,92782,463392,0,463465,897,0,14232,2566,521,623,1943,0,0,0,0,4,35751,120,2355,251995,663,580795,39,1,30,0,0,16,0,2,4,166,4243,934,39,298,0,0,1740,1821,13197,700246,0,700246,0,0,0,0,700246,700246,0,452,0,0,0,0,76,0,166777,0,0,0,8,428,3591,463465,340587,0,19,353765,0,416813,40267,0,2101,0,0,0,2938,27,24,38,52,0,0,0,0,60,0,6,0,0,0,0,0,12,0,0,2830,0,17694,18046,5,0,0,0,0,0,0,0,0,0,735,596,0,0,0,0,44
{siteid},2020-06-08,Admin,dept,515028,1247,202046,192033,12822,202046,0,202046,3486,0,71,3886,149,751,3150,0,0,0,0,0,92901,4375,131,497487,327,507224,42,0,0,0,0,42,0,0,0,0,146,403,0,435,0,0,0,0,40,514907,0,514907,0,0,0,0,514907,514907,0,183,0,0,0,0,0,0,366656,0,0,0,161,3310,3237,202046,141051,0,3739,128056,0,60395,166047,0,0,0,234,0,58,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4986,0,3564,3568,0,0,0,0,0,0,0,110,0,0,0,535,0,0,0,0,0
{siteid},2020-06-09,TDInternal,dept,39528,0,18748,13837,5342,18748,0,18748,3,0,48,551,548,548,3,0,0,0,0,47,6361,48,48,36463,193,38614,401,456,0,0,0,0,0,47,0,0,384,0,24,0,0,0,0,0,0,39528,0,39528,0,0,0,0,39528,39528,0,24,0,0,0,0,0,0,38440,0,0,0,720,35,83,18748,1521,0,47,599,0,9820,9978,0,0,0,0,0,98,0,0,0,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,307,0,7186,7238,0,0,0,0,0,0,0,0,0,0,0,115,0,0,0,0,0
{siteid},2020-06-07,TDInternal,dept,34189,0,16089,11929,4538,16089,0,16089,0,0,42,468,468,468,0,0,0,0,0,0,5367,42,42,31910,168,33563,311,399,0,0,0,0,0,0,0,0,336,0,21,0,0,0,0,0,0,34189,0,34189,0,0,0,0,34189,34189,0,21,0,0,0,0,0,0,33498,0,0,0,632,14,14,16089,1235,0,0,512,0,8280,8705,0,0,0,0,0,86,0,0,0,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,261,0,6100,6146,0,0,0,0,0,0,0,0,0,0,0,90,0,0,0,0,0
{siteid},2020-06-07,Admin,dept,380890,59,127289,120042,9000,127289,0,127289,1009,0,62,1530,92,332,1198,0,0,0,0,0,60512,3012,44,373431,536,376378,3,5,0,0,0,0,0,0,0,0,49,76,2,380,0,0,0,0,40,380890,0,380890,0,0,0,0,380890,380890,0,143,0,0,0,0,0,0,257194,0,0,0,85,2093,681,127289,108887,0,1400,102171,0,21156,119511,0,0,0,234,0,29,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1258,0,1121,1125,0,0,0,0,0,0,0,132,0,0,0,225,0,0,0,0,0
{siteid},2020-06-07,Unknown,Unknown,274738,220,101525,22618,79427,101525,0,101525,674,0,12874,2038,173,283,1755,0,0,0,0,0,14324,99,3676,190993,83,192944,

In [61]:
create volatile table FID as
(
Select Feature_ID 
from adlste_coa.coa_dim_feature f
where feature_bitpos <> 0
and f.DBS_Version = case when ( Select trim(substr(infoData,1,8)) as DBS_Version 
                                from dbc.dbcinfo where InfoKey = 'VERSION') = '16.20.40' 
                    then '16.20.40'
                    else 'DEFAULT' end
) with data 
 no primary index
 on commit preserve rows


Success: 0 rows affected

In [49]:
Select * from FeatureMap

Feature_Id,Feature_BitPos
24800,87
25700,111
29200,34
19000,126
21500,32
25200,91
23500,17
23400,72
30000,61
18800,37


In [68]:
Select * from FeatureMap
pivot 
(max(Feature_BitPos) as bitpos
 FOR Feature_ID in(
      Select cast(Feature_ID as integer) as Feature_ID 
      from FID where feature_ID between 10000 and 15000
 )
) tmp

11200_bitpos,14200_bitpos
59,58


In [60]:
Select * from FeatureMap
pivot 
(max(Feature_BitPos) as bitpos
,max(Feature_ID) as FID
 FOR Feature_ID in(
      11200,14200,17900,18000,18100,18200,18300,18400,18500,18600,18700,18800
     ,18900,19000,19100,19200,19300,19400,19500,19600,19700,19800,19900,20000
     ,20100,20200,20300,20400,20500,21100,21200,21300,21400,21500,21600,21700
     ,21800,21900,22000,22100,22200,22300,22400,22500,22600,22700,22800,22900
     ,23000,23100,23200,23300,23400,23500,23600,23700,23800,23900,24000,24100
     ,24200,24300,24400,24500,24600,24700,24800,24900,25000,25100,25200,25300
     ,25400,25500,25600,25700,25800,25900,26000,26100,26200,26300,26400,26500
     ,26600,26700,26800,26900,27000,27100,27200,27300,27400,27500,27600,27700
     ,27800,27900,28000,28100,28200,28300,28400,28500,28600,28700,28800,29100
     ,29200,29300,29400,29500,29600,29700,29800,29900,30000,30100,30200,30300
     ,30500
 )
) tmp

11200_bitpos,11200_FID,14200_bitpos,14200_FID,17900_bitpos,17900_FID,18000_bitpos,18000_FID,18100_bitpos,18100_FID,18200_bitpos,18200_FID,18300_bitpos,18300_FID,18400_bitpos,18400_FID,18500_bitpos,18500_FID,18600_bitpos,18600_FID,18700_bitpos,18700_FID,18800_bitpos,18800_FID,18900_bitpos,18900_FID,19000_bitpos,19000_FID,19100_bitpos,19100_FID,19200_bitpos,19200_FID,19300_bitpos,19300_FID,19400_bitpos,19400_FID,19500_bitpos,19500_FID,19600_bitpos,19600_FID,19700_bitpos,19700_FID,19800_bitpos,19800_FID,19900_bitpos,19900_FID,20000_bitpos,20000_FID,20100_bitpos,20100_FID,20200_bitpos,20200_FID,20300_bitpos,20300_FID,20400_bitpos,20400_FID,20500_bitpos,20500_FID,21100_bitpos,21100_FID,21200_bitpos,21200_FID,21300_bitpos,21300_FID,21400_bitpos,21400_FID,21500_bitpos,21500_FID,21600_bitpos,21600_FID,21700_bitpos,21700_FID,21800_bitpos,21800_FID,21900_bitpos,21900_FID,22000_bitpos,22000_FID,22100_bitpos,22100_FID,22200_bitpos,22200_FID,22300_bitpos,22300_FID,22400_bitpos,22400_FID,22500_bitpos,22500_FID,22600_bitpos,22600_FID,22700_bitpos,22700_FID,22800_bitpos,22800_FID,22900_bitpos,22900_FID,23000_bitpos,23000_FID,23100_bitpos,23100_FID,23200_bitpos,23200_FID,23300_bitpos,23300_FID,23400_bitpos,23400_FID,23500_bitpos,23500_FID,23600_bitpos,23600_FID,23700_bitpos,23700_FID,23800_bitpos,23800_FID,23900_bitpos,23900_FID,24000_bitpos,24000_FID,24100_bitpos,24100_FID,24200_bitpos,24200_FID,24300_bitpos,24300_FID,24400_bitpos,24400_FID,24500_bitpos,24500_FID,24600_bitpos,24600_FID,24700_bitpos,24700_FID,24800_bitpos,24800_FID,24900_bitpos,24900_FID,25000_bitpos,25000_FID,25100_bitpos,25100_FID,25200_bitpos,25200_FID,25300_bitpos,25300_FID,25400_bitpos,25400_FID,25500_bitpos,25500_FID,25600_bitpos,25600_FID,25700_bitpos,25700_FID,25800_bitpos,25800_FID,25900_bitpos,25900_FID,26000_bitpos,26000_FID,26100_bitpos,26100_FID,26200_bitpos,26200_FID,26300_bitpos,26300_FID,26400_bitpos,26400_FID,26500_bitpos,26500_FID,26600_bitpos,26600_FID,26700_bitpos,26700_FID,26800_bitpos,26800_FID,26900_bitpos,26900_FID,27000_bitpos,27000_FID,27100_bitpos,27100_FID,27200_bitpos,27200_FID,27300_bitpos,27300_FID,27400_bitpos,27400_FID,27500_bitpos,27500_FID,27600_bitpos,27600_FID,27700_bitpos,27700_FID,27800_bitpos,27800_FID,27900_bitpos,27900_FID,28000_bitpos,28000_FID,28100_bitpos,28100_FID,28200_bitpos,28200_FID,28300_bitpos,28300_FID,28400_bitpos,28400_FID,28500_bitpos,28500_FID,28600_bitpos,28600_FID,28700_bitpos,28700_FID,28800_bitpos,28800_FID,29100_bitpos,29100_FID,29200_bitpos,29200_FID,29300_bitpos,29300_FID,29400_bitpos,29400_FID,29500_bitpos,29500_FID,29600_bitpos,29600_FID,29700_bitpos,29700_FID,29800_bitpos,29800_FID,29900_bitpos,29900_FID,30000_bitpos,30000_FID,30100_bitpos,30100_FID,30200_bitpos,30200_FID,30300_bitpos,30300_FID,30500_bitpos,30500_FID
59,11200,58,14200,92,17900,93,18000,96,18100,94,18200,83,18300,26,18400,73,18500,49,18600,36,18700,37,18800,107,18900,126,19000,44,19100,108,19200,43,19300,50,19400,135,19500,38,19600,45,19700,41,19800,42,19900,121,20000,74,20100,75,20200,137,20300,136,20400,39,20500,95,21100,30,21200,31,21300,33,21400,32,21500,24,21600,23,21700,40,21800,20,21900,25,22000,120,22100,84,22200,131,22300,19,22400,102,22500,18,22600,130,22700,47,22800,46,22900,125,23000,124,23100,123,23200,122,23300,72,23400,17,23500,81,23600,76,23700,80,23800,71,23900,100,24000,99,24100,97,24200,57,24300,21,24400,118,24500,119,24600,86,24700,87,24800,90,24900,88,25000,85,25100,91,25200,89,25300,112,25400,113,25500,110,25600,111,25700,104,25800,105,25900,70,26000,116,26100,117,26200,115,26300,114,26400,128,26500,101,26600,127,26700,82,26800,109,26900,29,27000,28,27100,27,27200,134,27300,35,27400,103,27500,16,27600,78,27700,79,27800,53,27900,56,28000,98,28100,106,28200,54,28300,55,28400,51,28500,52,28600,133,28700,132,28800,48,29100,34,29200,129,29300,63,29400,67,29500,62,29600,66,29700,60,29800,65,29900,61,30000,69,30100,68,30200,64,30300,77,30500


In [81]:
/*  extracts  the feature logging from dbqlogtbl WITHOUT
     the cartesian Join

  parameters
     dbqlogtbl  = {dbqlogtbl}
     siteid     = {siteid}
     startdate  = {startdate}
     enddate    = {enddate}
*/

/*{{save:dbql_feature_logging.csv}}*/
SELECT
 '{siteid}' (VARCHAR(100)) as SiteID
,A.LogDate as LogDate
,u.User_Bucket
,u.User_Department
,count(*) as Request_Count
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -16)))) AS "Collect Statistics"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -17)))) AS "Fallback"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -18)))) AS "SET Table"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -19)))) AS "MULTISET Table"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -20)))) AS "Hashed Table"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -21)))) AS "Trigger"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -22)))) AS "Block Level Compression"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -23)))) AS "Auto Compression"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -24)))) AS "Teradata Columnar"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -25)))) AS "No Primary Index"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -26)))) AS "Row Partitioning"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -27)))) AS "Multi Level Partitioning"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -28)))) AS "8 Byte Partitioning"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -29)))) AS "2 Byte Partitioning"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -30)))) AS "Column Partitioning and Row Partitioning"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -31)))) AS "Column-Partitioned and a NoPI"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -32)))) AS "Column-Partitioned and a PI"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -33)))) AS "Column-Partitioned and a PA"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -34)))) AS "Teradata Temporal"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -35)))) AS "Parameterized Query"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -36)))) AS "Binary LOB"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -37)))) AS "Character LOB"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -38)))) AS "Small LOB"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -39)))) AS "Scalar Sub Query"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -40)))) AS "Statement Info Parcel"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -41)))) AS "User Defined Type"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -42)))) AS "XML Data Type"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -43)))) AS "JSON Data Type"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -44)))) AS "Distinct Data Type"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -45)))) AS "Structure Data Type"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -46)))) AS "Geospatial"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -47)))) AS "3D Geospatial"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -48)))) AS "Period Data Type"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -49)))) AS "Array Data Type"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -50)))) AS "Number Data Type"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -51)))) AS "Teradata Stored Procedure"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -52)))) AS "User Defined Function"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -53)))) AS "External Stored Procedure"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -54)))) AS "Table Function"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -55)))) AS "Table Operator"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -56)))) AS "Foreign Function"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -57)))) AS "Teradata Remote Query"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -58)))) AS "Query Grid"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -59)))) AS "Data Stream Architecture"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -60)))) AS "DBQL SQL Logging"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -61)))) AS "DBQL STEP Logging"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -62)))) AS "DBQL OBJECTS Logging"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -63)))) AS "DBQL EXPLAIN Logging"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -64)))) AS "DBQL XMLPLAN Logging"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -65)))) AS "DBQL STATSUSAGE Logging"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -66)))) AS "DBQL PARAMINFO Logging"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -67)))) AS "DBQL OBJECT USECOUNT Logging"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -68)))) AS "DBQL UTILITYINFO Logging"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -69)))) AS "DBQL THRESHOLD Enabled"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -70)))) AS "Incremental Planning and Execution"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -71)))) AS "Teradata Unity"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -72)))) AS "Autoreparse"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -73)))) AS "Teradata In-Memory"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -74)))) AS "Teradata Pivot"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -75)))) AS "Teradata Unpivot"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -76)))) AS "Redrive"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -77)))) AS "Queryband"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -78)))) AS "Index Analysis"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -79)))) AS "Partition Analysis"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -80)))) AS "Replication"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -81)))) AS "Load Isolation"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -82)))) AS "Single Sender Row Re-Distribution"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -83)))) AS "Partition Level Locking"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -84)))) AS "Primary Index"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -85)))) AS "Secondary Index"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -86)))) AS "Composite Secondary Index"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -87)))) AS "Join Index"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -88)))) AS "Non Unique UnHashed Index"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -89)))) AS "Unique UnHashed Index"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -90)))) AS "Non Unique Hashed Index"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -91)))) AS "Unique Hashed Index"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -92)))) AS "Fastexport Protocol"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -93)))) AS "Fastload Protocol"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -94)))) AS "Multiload Protocol"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -95)))) AS "Archive/Restore"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -96)))) AS "MloadX Protocol"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -97)))) AS "Fast Path Function"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -98)))) AS "R Table Operator"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -99)))) AS "Bit Manipulation Function"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -100)))) AS "Analytics Extensions"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -101)))) AS "Partial Redistribution Partial Duplication"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -102)))) AS "Queryable View Column Info"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -103)))) AS "Multiple Count Distinct"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -104)))) AS "Aggregation Push for Union All"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -105)))) AS "Binary Join Push for Union All"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -106)))) AS "Script Table Operator"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -107)))) AS "Dataset Data Type"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -108)))) AS "Dot Notation"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -109)))) AS "Union All Pushdown"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -110)))) AS "In-Memory Enhancement for Outer Join"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -111)))) AS "In-Memory Enhancement for PRPD Join"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -112)))) AS "In-Memory All Rows One AMP Optimization"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -113)))) AS "In-Memory AVX2 Support"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -114)))) AS "InList Rewrite to Subquery"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -115)))) AS "InList Rewrite to Outer Join"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -116)))) AS "InList Rewrite disqualifed by MultiTable Expression"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -117)))) AS "InList Rewrite Threshold Exceeded"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -118)))) AS "Sparse Map"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -119)))) AS "Two Maps"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -120)))) AS "Not System-Default Map"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -121)))) AS "LEAD LAG Analytics"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -122)))) AS "Geospatial Index Single Table Predicate"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -123)))) AS "Geospatial Index Single Table Distance Predicate"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -124)))) AS "Geospatial Index Join Predicate"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -125)))) AS "Geospatial Index Distance Join Predicate"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -126)))) AS "Dataset Data Type Storage Format CSV"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -127)))) AS "Push TOP N into UNION ALL Derived Tables"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -128)))) AS "In-Memory Bulk Qualification Enhancements"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -129)))) AS "Time Series Table"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -130)))) AS "Soft RetLimit for number of rows AMP step returns"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -131)))) AS "Function Alias"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -132)))) AS "QueryGrid Remote Tables"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -133)))) AS "QueryGrid Common Remote Table Elimination"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -134)))) AS "Dynamic Parameterization of Literals Feature"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -135)))) AS "SET TRANSFORM"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -136)))) AS "ODBC Scalar Functions"
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -137)))) AS "Multisource"
FROM PDCRINFO.DBQLOGTBL_HST as A
JOIN dim_user u
  on a.UserName = u.UserName
WHERE LogDate BETWEEN date-3 and date
GROUP BY 1,2,3,4;


SiteID,LogDate,User_Bucket,User_Department,Request_Count,Collect Statistics,Fallback,SET Table,MULTISET Table,Hashed Table,Trigger,Block Level Compression,Auto Compression,Teradata Columnar,No Primary Index,Row Partitioning,Multi Level Partitioning,8 Byte Partitioning,2 Byte Partitioning,Column Partitioning and Row Partitioning,Column-Partitioned and a NoPI,Column-Partitioned and a PI,Column-Partitioned and a PA,Teradata Temporal,Parameterized Query,Binary LOB,Character LOB,Small LOB,Scalar Sub Query,Statement Info Parcel,User Defined Type,XML Data Type,JSON Data Type,Distinct Data Type,Structure Data Type,Geospatial,3D Geospatial,Period Data Type,Array Data Type,Number Data Type,Teradata Stored Procedure,User Defined Function,External Stored Procedure,Table Function,Table Operator,Foreign Function,Teradata Remote Query,Query Grid,Data Stream Architecture,DBQL SQL Logging,DBQL STEP Logging,DBQL OBJECTS Logging,DBQL EXPLAIN Logging,DBQL XMLPLAN Logging,DBQL STATSUSAGE Logging,DBQL PARAMINFO Logging,DBQL OBJECT USECOUNT Logging,DBQL UTILITYINFO Logging,DBQL THRESHOLD Enabled,Incremental Planning and Execution,Teradata Unity,Autoreparse,Teradata In-Memory,Teradata Pivot,Teradata Unpivot,Redrive,Queryband,Index Analysis,Partition Analysis,Replication,Load Isolation,Single Sender Row Re-Distribution,Partition Level Locking,Primary Index,Secondary Index,Composite Secondary Index,Join Index,Non Unique UnHashed Index,Unique UnHashed Index,Non Unique Hashed Index,Unique Hashed Index,Fastexport Protocol,Fastload Protocol,Multiload Protocol,Archive/Restore,MloadX Protocol,Fast Path Function,R Table Operator,Bit Manipulation Function,Analytics Extensions,Partial Redistribution Partial Duplication,Queryable View Column Info,Multiple Count Distinct,Aggregation Push for Union All,Binary Join Push for Union All,Script Table Operator,Dataset Data Type,Dot Notation,Union All Pushdown,In-Memory Enhancement for Outer Join,In-Memory Enhancement for PRPD Join,In-Memory All Rows One AMP Optimization,In-Memory AVX2 Support,InList Rewrite to Subquery,InList Rewrite to Outer Join,InList Rewrite disqualifed by MultiTable Expression,InList Rewrite Threshold Exceeded,Sparse Map,Two Maps,Not System-Default Map,LEAD LAG Analytics,Geospatial Index Single Table Predicate,Geospatial Index Single Table Distance Predicate,Geospatial Index Join Predicate,Geospatial Index Distance Join Predicate,Dataset Data Type Storage Format CSV,Push TOP N into UNION ALL Derived Tables,In-Memory Bulk Qualification Enhancements,Time Series Table,Soft RetLimit for number of rows AMP step returns,Function Alias,QueryGrid Remote Tables,QueryGrid Common Remote Table Elimination,Dynamic Parameterization of Literals Feature,SET TRANSFORM,ODBC Scalar Functions,Multisource
{siteid},2020-06-10,Unknown,Unknown,690325,192,474302,375552,91768,474316,0,474479,2791,0,18585,3090,990,1077,2013,0,0,0,0,97,24059,171,3843,199742,1268,486635,113,4,14,0,0,83,0,84,4,478,2809,662,2,564,0,0,1887,2009,15790,690295,0,690295,0,0,0,0,690295,690295,0,414,0,0,0,0,15,0,144202,0,0,0,6,638,4347,474479,300882,0,89,316505,0,380469,92246,0,1511,0,0,0,2860,1,12,67,101,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0,5974,0,49641,50203,65,0,0,0,0,0,0,0,0,0,963,657,0,0,0,0,71
{siteid},2020-06-08,TDInternal,dept,39519,0,18569,13733,5268,18569,0,18569,0,0,48,553,553,553,0,0,0,0,0,0,6270,48,48,36356,192,38678,356,456,0,0,0,0,0,0,0,0,384,0,24,0,0,0,0,0,0,39519,0,39519,0,0,0,0,39519,39519,0,24,0,0,0,0,0,0,38324,0,0,0,720,251,251,18569,1465,0,0,606,0,9653,9969,0,0,0,0,0,98,0,0,0,36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,308,0,7103,7155,0,0,0,0,0,0,0,0,0,0,0,110,0,0,0,0,0
{siteid},2020-06-08,Unknown,Unknown,700246,230,463386,371935,92782,463392,0,463465,897,0,14232,2566,521,623,1943,0,0,0,0,4,35751,120,2355,251995,663,580795,39,1,30,0,0,16,0,2,4,166,4243,934,39,298,0,0,1740,1821,13197,700246,0,700246,0,0,0,0,700246,700246,0,452,0,0,0,0,76,0,166777,0,0,0,8,428,3591,463465,340587,0,19,353765,0,416813,40267,0,2101,0,0,0,2938,27,24,38,52,0,0,0,0,60,0,6,0

In [ ]:
CREATE MULTISET GLOBAL TEMPORARY TABLE ADLSTE_coa_stg.stg_dat_level1_IO_Forecast 
(
    SiteID VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
    "Report Date" VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
    "Log Date" VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
    "Peak Start" VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
    "Peak End" VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
    "Avg I/O Pct" VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
    "Moving Avg" VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
    Trend VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
    ReserveX BIGINT,
    "Reserve Horizon" VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
    SlopeX VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC)
    PRIMARY INDEX ( SiteID ,"Report Date" ,"Log Date" 
)
ON COMMIT PRESERVE ROWS;

In [ ]:
CREATE SET TABLE ADLSTE_coa.coat_dat_level1_IO_Forecast  
(
    SiteID VARCHAR(30) CHARACTER SET LATIN NOT CASESPECIFIC NOT NULL,
    "Report Date" DATE FORMAT 'YYYY-MM-DD' NOT NULL,
    "Log Date" DATE FORMAT 'YYYY-MM-DD' NOT NULL,
    "Peak Start" VARCHAR(30) CHARACTER SET LATIN NOT CASESPECIFIC,
    "Peak End" VARCHAR(30) CHARACTER SET LATIN NOT CASESPECIFIC,
    "Avg IO Pct" DECIMAL(18,4),
    "Moving Avg" DECIMAL(18,4),
    Trend DECIMAL(18,4),
    ReserveX DECIMAL(18,4),
    "Reserve Horizon" DECIMAL(18,4),
    SlopeX DECIMAL(18,4)
)
UNIQUE PRIMARY INDEX ( SiteID ,"Report Date" ,"Log Date" );

In [ ]:
replace view  ADLSTE_COA.coa_dat_level1_IO_Forecast as locking row for access
Select * from ADLSTE_COA.coat_dat_level1_IO_Forecast

In [ ]:
-- insert into ADLSTE_COA.coat_dat_level1_IO_Forecast Select * from ADLSTE_WestComm.Consumption_IO_Forecast_V2;

In [ ]:
REPLACE PROCEDURE adlste_coa.sp_dat_level1_IO_Forecast ( fileset_version VARCHAR(128) )
SQL SECURITY CREATOR
BEGIN
  --call sp_audit_Account_Name ('adlste_coa.stg_dat_dbcinfo', 'sp_dat_dbcinfo', '{}');
  --call sp_audit_Site_ID ('adlste_coa.stg_dat_dbcinfo', 'sp_dat_dbcinfo', '{}');

  delete from ADLSTE_COA.coa_dat_level1_IO_Forecast
  where   (SiteID, "Log Date") in
   (Select SiteID, "Log Date" From ADLSTE_COA_Stg.stg_dat_level1_IO_Forecast);

  Insert into ADLSTE_COA.coa_dat_level1_IO_Forecast
  Select * From ADLSTE_COA_Stg.stg_dat_level1_IO_Forecast;
END;


In [ ]:
call adlste_coa.sp_dat_level1_IO_Forecast('v2')

In [ ]:
Select top 10 * 
from adlste_coa.coa_dat_level1_IO_Forecast
where siteid like 'BOEING%'

<br/><br/><br/>
## Storage Trend

In [ ]:
drop table ADLSTE_coa_stg.stg_dat_level1_Storage_Forecast

In [ ]:
CREATE MULTISET GLOBAL TEMPORARY TABLE ADLSTE_coa_stg.stg_dat_level1_Storage_Forecast 
(
      SiteID VARCHAR(30) CHARACTER SET LATIN NOT CASESPECIFIC NOT NULL,
      "Report Date" DATE FORMAT 'YYYY-MM-DD' NOT NULL,
      "Log Date" DATE FORMAT 'YYYY-MM-DD' NOT NULL,
      "Total Max Perm" BIGINT,
      "Total Current Perm" BIGINT,
      "Total Peak Perm" BIGINT,
      "Total Available Perm" BIGINT,
      "Total Current Pct" DECIMAL(18,4),
      "Total Available Pct" DECIMAL(18,4),
      "Moving Avg" DECIMAL(18,4),
      Trend DECIMAL(18,4),
      ReserveX DECIMAL(18,4),
      "Reserve Horizon" DECIMAL(18,4),
      SlopeX DECIMAL(18,4)
)
ON COMMIT PRESERVE ROWS;

In [ ]:
CREATE SET TABLE ADLSTE_coa.coat_dat_level1_Storage_Forecast ,FALLBACK ,
     NO BEFORE JOURNAL,
     NO AFTER JOURNAL,
     CHECKSUM = DEFAULT,
     DEFAULT MERGEBLOCKRATIO
     (
      SiteID VARCHAR(30) CHARACTER SET LATIN NOT CASESPECIFIC NOT NULL,
      "Report Date" DATE FORMAT 'YYYY-MM-DD' NOT NULL,
      "Log Date" DATE FORMAT 'YYYY-MM-DD' NOT NULL,
      "Total Max Perm" BIGINT,
      "Total Current Perm" BIGINT,
      "Total Peak Perm" BIGINT,
      "Total Available Perm" BIGINT,
      "Total Current Pct" DECIMAL(18,4),
      "Total Available Pct" DECIMAL(18,4),
      "Moving Avg" DECIMAL(18,4),
      Trend DECIMAL(18,4),
      ReserveX DECIMAL(18,4),
      "Reserve Horizon" DECIMAL(18,4),
      SlopeX DECIMAL(18,4))
UNIQUE PRIMARY INDEX ( SiteID ,"Report Date" ,"Log Date" );

In [ ]:
insert into ADLSTE_COA.coat_dat_level1_Storage_Forecast
Select * from ADLSTE_WestComm.Consumption_Storage_Forecast_V2;

In [ ]:
replace view  ADLSTE_COA.coa_dat_level1_Storage_Forecast as locking row for access
Select * from ADLSTE_COA.coat_dat_level1_Storage_Forecast

In [ ]:
REPLACE PROCEDURE adlste_coa.sp_dat_level1_Storage_Forecast ( fileset_version VARCHAR(128) )
SQL SECURITY CREATOR
BEGIN
  --call sp_audit_Account_Name ('adlste_coa.stg_dat_dbcinfo', 'sp_dat_dbcinfo', '{}');
  --call sp_audit_Site_ID ('adlste_coa.stg_dat_dbcinfo', 'sp_dat_dbcinfo', '{}');

  delete from ADLSTE_COA.coa_dat_level1_Storage_Forecast
  where   (SiteID, "Log Date") in
   (Select SiteID, "Log Date" From ADLSTE_COA_Stg.stg_dat_level1_Storage_Forecast);

  Insert into ADLSTE_COA.coa_dat_level1_Storage_Forecast
  Select * From ADLSTE_COA_Stg.stg_dat_level1_Storage_Forecast;
END;


In [ ]:
call adlste_coa.sp_dat_level1_Storage_Forecast('v2')

In [ ]:
Select top 10 * 
from adlste_coa.coa_dat_level1_Storage_Forecast
where siteid like 'BOEING%'

<br><br><br><br>
# Test Process

In [ ]:
%connect name=TranscendAWS

###  Built Temp Table:   dim_users

In [74]:

CREATE MULTISET VOLATILE TABLE "dim_user.csv"
("SiteID"                      VARCHAR(111)  CHARACTER SET UNICODE 
,"Pattern_Type"                VARCHAR(105)  CHARACTER SET UNICODE 
,"Pattern"                     VARCHAR(116)  CHARACTER SET UNICODE 
,"User_Bucket"                 VARCHAR(110)  CHARACTER SET UNICODE 
,"User_Department"             VARCHAR(104)  CHARACTER SET UNICODE 
,"User_SubDepartment"          VARCHAR(107)  CHARACTER SET UNICODE 
,"User_Region"                 VARCHAR(103)  CHARACTER SET UNICODE 
,"Priority"                    BIGINT
) NO PRIMARY INDEX
ON COMMIT PRESERVE ROWS;

Success: 0 rows affected

In [75]:

INSERT INTO "dim_user.csv"
SELECT
  cast('Default' as varchar(111))
 ,cast('Equal' as varchar(105))
 ,cast('Viewpoint' as varchar(116))
 ,cast('TDInternal' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(100 as bigint)
from (sel 1 one) i0    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Equal' as varchar(105))
 ,cast('TDWM' as varchar(116))
 ,cast('TDInternal' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(100 as bigint)
from (sel 1 one) i1    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Equal' as varchar(105))
 ,cast('CRASHDUMPS' as varchar(116))
 ,cast('TDInternal' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(100 as bigint)
from (sel 1 one) i2    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Equal' as varchar(105))
 ,cast('DBC' as varchar(116))
 ,cast('TDInternal' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(100 as bigint)
from (sel 1 one) i3    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Equal' as varchar(105))
 ,cast('DBMON' as varchar(116))
 ,cast('TDInternal' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(100 as bigint)
from (sel 1 one) i4    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('%STG%USER%' as varchar(116))
 ,cast('ETL' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(2100 as bigint)
from (sel 1 one) i5    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('BAR_%' as varchar(116))
 ,cast('Admin' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(2125 as bigint)
from (sel 1 one) i6    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('%_BAR' as varchar(116))
 ,cast('Admin' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(2150 as bigint)
from (sel 1 one) i7    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('DSA_%' as varchar(116))
 ,cast('Admin' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(2175 as bigint)
from (sel 1 one) i8    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('%_DSA' as varchar(116))
 ,cast('Admin' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(2200 as bigint)
from (sel 1 one) i9    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('ARC_%' as varchar(116))
 ,cast('Admin' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(2225 as bigint)
from (sel 1 one) i10    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('%_ARC' as varchar(116))
 ,cast('Admin' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(2250 as bigint)
from (sel 1 one) i11    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('DBADMIN%' as varchar(116))
 ,cast('Admin' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(2500 as bigint)
from (sel 1 one) i12    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('%_ADMIN' as varchar(116))
 ,cast('Admin' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(2600 as bigint)
from (sel 1 one) i13    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('%_DBA' as varchar(116))
 ,cast('Admin' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(2800 as bigint)
from (sel 1 one) i14    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('CALLREP%' as varchar(116))
 ,cast('Analytic' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(4300 as bigint)
from (sel 1 one) i15    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('%Tableau%' as varchar(116))
 ,cast('Analytic' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(4400 as bigint)
from (sel 1 one) i16    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('%BOBJ%' as varchar(116))
 ,cast('Analytic' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(4500 as bigint)
from (sel 1 one) i17    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('%_ETL_%' as varchar(116))
 ,cast('ETL' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(6000 as bigint)
from (sel 1 one) i18    UNION ALL
SELECT
  cast('TDCLOUD%TD%' as varchar(111))
 ,cast('RegEx' as varchar(105))
 ,cast('[a-z]{2}[0-9]{6}' as varchar(116))
 ,cast('Analytic' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(7000 as bigint)
from (sel 1 one) i19    ;

Success: 20 rows affected

In [76]:

create volatile table dim_user as
(
  select
   '{siteid}' as Site_ID
  ,o.UserName
  ,o.UserHash
  ,coalesce(p.User_Bucket,'Unknown') as User_Bucket
  ,coalesce(p.User_Department, 'Unknown') as User_Department
  ,coalesce(p.User_SubDepartment, 'Unknown') as User_SubDepartment
  ,coalesce(p.User_Region, 'Unknown') as User_Region
  ,coalesce(p.Priority,1e6) as Priority_
  ,coalesce(p.Pattern_Type,'Equal')  as Pattern_Type
  ,coalesce(p.Pattern, o.UserName) as Pattern
  ,coalesce(p.SiteID, 'None')        as SiteID_
  from (select
         trim(DatabaseName) as UserName
        ,substr(Username,1,3) as first3
        ,substr(Username,floor(character_length(Username)/2)-1,3) as middle3
        ,substr(Username,character_length(Username)-3,3) as last3
        /* generate UserHash value */
        ,trim(cast(from_bytes(hashrow( Username),'base16') as char(9))) ||
         trim(cast(from_bytes(hashrow( first3  ),'base16') as char(9))) ||
         trim(cast(from_bytes(hashrow( middle3 ),'base16') as char(9))) ||
         trim(cast(from_bytes(hashrow( last3   ),'base16') as char(9))) as UserHash
        from dbc.DatabasesV where DBKind = 'U'
        ) as o
  left join "dim_user.csv" as p
    on (case
        when p.Pattern_Type = 'Equal' and o.UserName = p.Pattern then 1
        when p.Pattern_Type = 'Like'  and o.UserName like p.Pattern then 1
        when p.Pattern_Type = 'RegEx'
         and character_length(regexp_substr(o.UserName, p.Pattern,1,1,'i'))>0 then 1
        else 0 end) = 1
    and SiteID_ in('default','None') or '{siteid}' like SiteID_
  qualify Priority_ = min(Priority_)over(partition by o.UserName)
) with data
primary index (UserName)
on commit preserve rows
;


Success: 0 rows affected

In [77]:
drop table "dim_user.csv"
;

Success: 0 rows affected

In [78]:
collect stats on dim_user column(UserName)
;

Success: 0 rows affected

In [79]:
/*{{save:all_users.csv}}*/
Select UserName, UserHash, User_Bucket
,User_Department, User_SubDepartment, User_Region
from dim_user
;

UserName,UserHash,User_Bucket,User_Department,User_SubDepartment,User_Region
sw186033,4D9FFF2-BFF7A1972E40B-717B8F63,Unknown,Unknown,Unknown,Unknown
RK185080,EC0ED00-FAB566757BD8F52-788FCF99,Unknown,Unknown,Unknown,Unknown
NA250053,4D3566B7FBEB04CADCA16B206CDD6,Unknown,Unknown,Unknown,Unknown
yg186000,4D34A34-7CF6570972E40B202CD97E,Unknown,Unknown,Unknown,Unknown
JT230019,8114D7D663371CC-3E9719A551A8885B,Unknown,Unknown,Unknown,Unknown
bs186041,A6E055F51005B3C972E40B-64614187,Unknown,Unknown,Unknown,Unknown
SI250014,3E5F8BE7FF09857ADCA16B51A8885B,Unknown,Unknown,Unknown,Unknown
AA250132,1F80D149-2E26E8C2ADCA16B28A504F8,Unknown,Unknown,Unknown,Unknown
MN186041,52FAA09-190D946D972E40B-64614187,Unknown,Unknown,Unknown,Unknown
AG186089,9BD4AA2166F84FC972E40B-127FE,Unknown,Unknown,Unknown,Unknown
